In [9]:
import os
import pandas as pd

In [2]:
train_dir = '/opt/ml/input/data/train/images_augmented_1000'

In [17]:
gender = 'female'
gender = 0 if gender == 'male' else 1
gender

1

In [25]:
profiles = os.listdir(train_dir)
indices = []
for profile in profiles:
    if profile.startswith('.'):
        continue
    
    id, gender, race, age = profile.split('_')
    gender = 0 if gender == 'male' else 1
    
    names = os.listdir(os.path.join(train_dir, profile))
    for name in names:
        if name.startswith('.'):
            continue
        
        if name.startswith('mask'):
            mask = 0
        elif name.startswith('in'):
            mask = 1
        else:
            mask = 2
        name_path = os.path.join(train_dir, profile, name)

        indices.append((name_path, gender, age, mask))
# gender: male -> 0, female -> 1
# mask: mask -> 0, incorrect -> 1, normal -> 2
profiles_df = pd.DataFrame(indices, columns=['path', 'gender', 'age', 'mask'])
profiles_df.path[0]





'/opt/ml/input/data/train/images_augmented_1000/3021001640_male_Asian_18/incorrect_mask3.jpg'

In [28]:
profiles_df.age.value_counts().index.max(), profiles_df.age.value_counts().index.min()

('60', '18')

In [ ]:
best_test_accuracy = 0.
best_test_loss = 9999.
for epoch in range(NUM_EPOCH):
  running_loss = 0.
  running_acc = 0.
  net_gender.train()
  for ind, (images, labels) in enumerate(tqdm(gender_dataloader)):
    images = images.to(device)
    labels = labels.to(device)
    optim_gender.zero_grad() # parameter gradient를 업데이트 전 초기화함
    logits = net_gender(images)
    _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함 
    loss = loss_fn(logits, labels.long())
    loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
    optim_gender.step() # 계산된 gradient를 가지고 모델 업데이트
    running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
    running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
  # 한 epoch이 모두 종료되었을 때,
  epoch_loss = running_loss / len(gender_dataloader.dataset)
  epoch_acc = running_acc / len(gender_dataloader.dataset)
  print(f"현재 epoch-{epoch}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
  if best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
    best_test_accuracy = epoch_acc
  if best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
    best_test_loss = epoch_loss
print("gender 학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")
torch.save(net_gender.state_dict(), f"{save_dir}/gender.pth")